In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import keras
import matplotlib.pyplot as plt
import utm
from keras import backend as K

def distance_loss(y_pred, y_true):
    return K.sqrt(K.mean(K.sum(K.square(y_pred-y_true), axis=-1)))

def median_absolute_deviation(y_pred, y_true):
    deviation = np.abs(y_pred,y_true)
    return np.mean(deviation, axis=0)

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch': [], 'epoch': []}
        self.accuracy = {'batch': [], 'epoch': []}
        self.val_loss = {'batch': [], 'epoch': []}
        self.val_acc = {'batch': [], 'epoch': []}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()


data = pd.read_csv('hwphase2data_update/new_formatted_data2.csv')

X = [
    np.array([[mr[4:9],
              mr[9:14],
              mr[14:19],
              mr[19:24],
              mr[24:29],
              mr[29:34]]]).T for mr in data.values
]
X = np.array(X)
y = []
zone_number = 0
zone_letter = 0
for mr in data.values:
    lat, lon, zone_number, zone_letter = utm.from_latlon(mr[-4], mr[-5])
    y.append([lat, lon])
scaler = MinMaxScaler()
y = np.array(y)
scaler.fit(y)
y = scaler.transform(y)

def build_model():
    m = Sequential()
    m.add(Conv2D(32, kernel_size=2, activation='relu', input_shape=(5, 6, 1)))
    m.add(Conv2D(16, kernel_size=2, activation='relu'))
    m.add(Flatten())
    m.add(Dense(1024, activation='relu'))
    m.add(Dropout(0.2))
    m.add(Dense(512, activation='relu'))
    m.add(Dropout(0.2))
    m.add(Dense(256, activation='relu'))
    m.add(Dense(2))
    r = optimizers.Adam(lr=0.001,decay=0.00001)
    m.compile(optimizer=r, loss=distance_loss, metrics=['mse'])

    return m


history = LossHistory()
X_train, X_test, y_train, y_test = train_test_split(X,y)
model = build_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=500, batch_size=64, callbacks=[history])
history.loss_plot('epoch')
# data_test = pd.read_csv('new_formatted_.csv')
# X_test = [
#     np.array([[mr[4:9],
#               mr[9:14],
#               mr[14:19],
#               mr[19:24],
#               mr[24:29],
#               mr[29:34]]]).T for mr in data_test.values
# ]
# ------
X_test = np.array(X_test)
y_pred = model.predict(X_test)
y_pred = y_pred.reshape(-1,2)
y_pred = scaler.inverse_transform(y_pred)
print(distance_loss(y_pred,scaler.inverse_transform(y_test)))
# ------
# y_final = [[utm.to_latlon(i[0], i[1], zone_number=zone_number, zone_letter=zone_letter)[1],
#             utm.to_latlon(i[0], i[1], zone_number=zone_number, zone_letter=zone_letter)[0]] for i in y_pred]
# y_final = np.array(y_final)
# df_pred = pd.DataFrame(data={'Longitude':y_final[:,0], 'Latitude':y_final[:,1]})
# df_pred.to_csv('pred1.csv', index=False)

Train on 7535 samples, validate on 2512 samples
Epoch 1/500
7535/7535 [==============================] - 4s 556us/step - loss: 0.3048 - mean_squared_error: 0.0489 - val_loss: 0.2660 - val_mean_squared_error: 0.0356
Epoch 2/500
7535/7535 [==============================] - 3s 356us/step - loss: 0.2303 - mean_squared_error: 0.0268 - val_loss: 0.2011 - val_mean_squared_error: 0.0203
Epoch 3/500
7535/7535 [==============================] - 2s 315us/step - loss: 0.1946 - mean_squared_error: 0.0191 - val_loss: 0.1716 - val_mean_squared_error: 0.0148
Epoch 4/500
7535/7535 [==============================] - 3s 354us/step - loss: 0.1736 - mean_squared_error: 0.0152 - val_loss: 0.1766 - val_mean_squared_error: 0.0156
Epoch 5/500
7535/7535 [==============================] - 2s 276us/step - loss: 0.1701 - mean_squared_error: 0.0146 - val_loss: 0.1527 - val_mean_squared_error: 0.0117
Epoch 6/500
7535/7535 [==============================] - 2s 251us/step - loss: 0.1582 - mean_squared_error: 0.0126 - 

7535/7535 [==============================] - 2s 217us/step - loss: 0.0559 - mean_squared_error: 0.0016 - val_loss: 0.0777 - val_mean_squared_error: 0.0031
Epoch 50/500
7535/7535 [==============================] - 2s 202us/step - loss: 0.0558 - mean_squared_error: 0.0016 - val_loss: 0.0635 - val_mean_squared_error: 0.0021
Epoch 51/500
7535/7535 [==============================] - 1s 197us/step - loss: 0.0547 - mean_squared_error: 0.0015 - val_loss: 0.0690 - val_mean_squared_error: 0.0025
Epoch 52/500
7535/7535 [==============================] - 2s 208us/step - loss: 0.0553 - mean_squared_error: 0.0016 - val_loss: 0.0662 - val_mean_squared_error: 0.0023
Epoch 53/500
7535/7535 [==============================] - 2s 214us/step - loss: 0.0546 - mean_squared_error: 0.0015 - val_loss: 0.0725 - val_mean_squared_error: 0.0027
Epoch 54/500
7535/7535 [==============================] - 1s 195us/step - loss: 0.0566 - mean_squared_error: 0.0016 - val_loss: 0.0622 - val_mean_squared_error: 0.0020
Epoch

KeyboardInterrupt: 

In [ ]:
22 23